<a href="https://colab.research.google.com/github/Aterg264/Aterg264/blob/main/Cuotes_OpenCon_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Documentació](https://docs.google.com/document/d/1bG0x6nOVrm2-CRQS-PMozXm3yTAaRPpYMzMkbyemt0w/edit?tab=t.0)



In [85]:
import pandas as pd
from google.colab import files

Llegir l'excel amb les dades
Gènere:  Dona/Mujer, Home/Hombre, No-binari/No binario, Altre/Otro

In [86]:
uploaded = files.upload()

Saving test_open.csv to test_open (4).csv


In [87]:
respostes_totals = pd.read_csv("test_open.csv")
respostes_copia = respostes_totals
# print(respostes_totals.info())

In [88]:
columnes = ["id","Nomsentit", "Pronoms", "Pronoms1", "Nomregistral", "Poblacio",
            "Paisvalencia", "Edat", "Correo", "Telf", "Eixos", "Titoleixos",
            "Nomono", "Nomono1", "Genere", "Genere1", "Descgenere",
            "Genere Assignat","Genere Assignat1", "Hetero", "Hetero1", "Blanca",
            "Blanca1", "Procedencia", "Procedencia1", "Descgordo", "Gordofobia",
            "Gordofobia1", "Diska", "Diska1", "Descneuro",
            "Neurodiver", "Neurodiver1", "Llengua", "Descllengua", "Llengua1",
            "Edadisme", "Edadisme1", "Pobressa", "Pobressa1", "Descviogen",
            "Viogen", "Viogen1", "Treballsex", "Treballsex1", "Enfcronica",
            "Enfcronica1", "Rural", "Rural1", "Responsable", "Responsable1",
            "Altres", "Altresopresions"]
print(len(columnes))

53


In [89]:
respostes_totals.columns = columnes
# print(respostes_totals.head())

Crear una nova columna per la macrocuota de gènere

In [90]:
respostes_totals['Macrogenere'] = respostes_totals.apply(
    lambda row: 'No' if (row['Genere'] == 'Home' or row['Genere'] == 'Hombre')
    and (row['Genere Assignat'] == 'Sí' or row['Genere Assignat'] == 'Si')
    else 'Si', axis=1)

respostes_totals['Xenofobia'] = respostes_totals.apply(
    lambda row: 'Si' if (row['Blanca'] == 'No')
    or (row['Procedencia'] == 'Sí' or row['Procedencia'] == 'Si')
    else 'Si', axis=1)

In [91]:
respostes_totals['Hetero'] = respostes_totals.apply(
    lambda row: 'No' if (row['Hetero'] == 'Sí' or row['Hetero'] == 'Si')
    else 'Si', axis=1)

respostes_totals['Genere Assignat'] = respostes_totals.apply(
    lambda row: 'No' if (row['Genere Assignat'] == 'Sí' or row['Genere Assignat'] == 'Si')
    else 'Si', axis=1)

### Calcular cuotes



**Macrocuotes**


In [92]:
resp_macro = respostes_totals[["Paisvalencia", 'Macrogenere']]
resp_micromacro = respostes_totals[[
    "id", "Paisvalencia", 'Macrogenere', "Genere Assignat", "Hetero",
    "Xenofobia", "Gordofobia", "Diska", "Neurodiver", "Llengua",
    "Edadisme", "Pobressa", "Viogen", "Treballsex", "Enfcronica", "Rural",
    "Responsable", "Altres"]]

**Minicuotes**

Modificar el dataser perque tots els "Si" siguin sense accent i ordenar les columnes per número de Si. Després sumar els "Si" per ordenar les cuotes de menys a més.

In [93]:
col_mini = [
    "Genere Assignat", "Hetero", "Xenofobia", "Gordofobia", "Diska",
    "Neurodiver", "Llengua", "Edadisme", "Pobressa", "Viogen", "Treballsex",
    "Enfcronica", "Rural", "Responsable","Altres"]
resp_mini = respostes_totals[col_mini]

resp_macro = resp_macro.map(lambda x: 'Si' if x == 'Sí' else x)
resp_mini = resp_mini.map(lambda x: 'Si' if x == 'Sí' else x)
resp_micromacro = resp_micromacro.map(lambda x: 'Si' if x == 'Sí' else x)


ordre_cuotes = resp_mini.apply(lambda x: (x == 'Si').sum(), axis=0)
ordre_cuotes = ordre_cuotes.sort_values(ascending=True).index.tolist()

print(ordre_cuotes)

['Diska', 'Treballsex', 'Responsable', 'Altres', 'Genere Assignat', 'Edadisme', 'Pobressa', 'Rural', 'Enfcronica', 'Gordofobia', 'Viogen', 'Neurodiver', 'Hetero', 'Llengua', 'Xenofobia']


**Càlculs**

En la cuota hi ha menys o igual al nombre de persones que poden entrar per cada cuota.

In [94]:
# from re import L
# Modificar aquestes dades si es modifica el total
total = 94

total_cuota = 6
nomixte = 2
pais_valencia = 2
altres = 2

resp_cuotes_open = resp_micromacro.copy()
resp_cuotes_atzar = resp_micromacro.copy()
resp_cuotes_open["Open"] = "No"

for cuota in ordre_cuotes:

  # Cuotes en les que hi ha menys de 6 persones
  if ((resp_cuotes_atzar[cuota] == 'Si').sum()) <= 6:

    print(cuota)
    print(list(resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si', "id"]))

    resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si', 'Open'] = "Si"
    resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"

  # Cuotes en les que hi ha més de 6 persones
  else:
    n_cuota_si = 2

    # Valencia
    cuota_si = resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si']
    cuota_si_valencia = cuota_si.loc[cuota_si["Paisvalencia"] == 'Si']

    if len(cuota_si_valencia) >= pais_valencia:
      cuota_si_valencia = cuota_si_valencia.sample(n=pais_valencia)
    elif len(cuota_si_valencia) == 0:
      n_cuota_si += 2
    else:
      n_cuota_si += 1
    resp_cuotes_open.loc[resp_cuotes_open["id"].isin(cuota_si_valencia["id"]),'Open'] = "Si"
    resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"

    print(f"Minicuota: {cuota}, Macrocuota: Païs valencià")
    print(list(cuota_si_valencia["id"]))

    # No mixte
    cuota_si = resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si']
    cuota_si_nomixte = cuota_si.loc[cuota_si["Macrogenere"] == 'Si']

    if len(cuota_si_nomixte) >= nomixte:
      cuota_si_nomixte = cuota_si_nomixte.sample(n=nomixte)
    elif len(cuota_si_nomixte) == 0:
      n_cuota_si += 2
    else:
      n_cuota_si += 1
    resp_cuotes_open.loc[resp_cuotes_open["id"].isin(cuota_si_nomixte["id"]),'Open'] = "Si"
    resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"

    print(f"Minicuota: {cuota}, Macrocuota: No mixte")
    print(list(cuota_si_nomixte["id"]))

    # Orga
    cuota_si = resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si']

    cuota_si_orga = cuota_si[cuota_si["id"].str.len() == 1 &
                          cuota_si["id"].isin(list("12345678"))]
    if len(cuota_si_orga) >= n_cuota_si:
      cuota_si_orga = cuota_si_orga.sample(n=n_cuota_si)
    n_cuota_si -= len(cuota_si_orga)
    resp_cuotes_open.loc[resp_cuotes_open["id"].isin(cuota_si_orga["id"]),'Open'] = "Si"
    resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"

    print(f"Minicuota: {cuota} orga")
    print(list(cuota_si_orga["id"]))

    # La resta
    cuota_si_resta = resp_cuotes_open.loc[resp_cuotes_atzar[cuota] == 'Si']

    if n_cuota_si>0:
      if cuota!="Altres":
        if len(cuota_si_resta) >= n_cuota_si:
          cuota_si_resta = cuota_si_resta.sample(n=n_cuota_si)
      elif n_cuota_si>1:
        cuota_si_resta = cuota_si_resta.sample(n=n_cuota_si-1)

      resp_cuotes_open.loc[resp_cuotes_open["id"].isin(cuota_si_resta["id"]),'Open'] = "Si"
      resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"
      print(f"Minicuota: {cuota} total")
      print(list(cuota_si_resta["id"]))
    else:
      print([])



Diska
['M', 'UB', 'YC']
Minicuota: Treballsex, Macrocuota: Païs valencià
['UC', 'IC']
Minicuota: Treballsex, Macrocuota: No mixte
['LC', 'LB']
Minicuota: Treballsex orga
[]
Minicuota: Treballsex total
['KB', 'Z']
Minicuota: Responsable, Macrocuota: Païs valencià
['HC', 'GD']
Minicuota: Responsable, Macrocuota: No mixte
['FD', 'IB']
Minicuota: Responsable orga
[]
Minicuota: Responsable total
['DD', 'DC']
Minicuota: Altres, Macrocuota: Païs valencià
['HD', 'XB']
Minicuota: Altres, Macrocuota: No mixte
['OC', 'AA']
Minicuota: Altres orga
[]
Minicuota: Altres total
['TB']
Minicuota: Genere Assignat, Macrocuota: Païs valencià
['KC', 'TC']
Minicuota: Genere Assignat, Macrocuota: No mixte
['BD', '6']
Minicuota: Genere Assignat orga
['5']
Minicuota: Genere Assignat total
['PA']
Minicuota: Edadisme, Macrocuota: Païs valencià
['BA', 'RC']
Minicuota: Edadisme, Macrocuota: No mixte
['EC', 'R']
Minicuota: Edadisme orga
['2', '7']
[]
Minicuota: Pobressa, Macrocuota: Païs valencià
['WC', 'AC']
Minicu

In [95]:
totalf = total - ((resp_cuotes_open["Open"] == 'Si').sum())
print(f"Falten {totalf} persones per acabar les inscripcions")

Falten 8 persones per acabar les inscripcions


Calcular les places que faltin sense cuota

In [96]:
cuota_totis = resp_cuotes_open.loc[resp_cuotes_open["Open"] == "No"]
cuota_totis = cuota_totis.sample(n= totalf)
resp_cuotes_open.loc[resp_cuotes_open["id"].isin(cuota_totis["id"]),'Open'] = "Si"
resp_cuotes_atzar.loc[resp_cuotes_open["Open"] == 'Si', col_mini] = "No"

print(f"Sense cuota")
print(list(cuota_totis["id"]))

Sense cuota
['JC', 'YB', 'X', 'NB', 'OD', 'FB', 'OA', 'MA']


### Testejar si va bé

Comprobar si hi ha algun nom repetit

In [97]:
repe = resp_cuotes_open.loc[resp_cuotes_open["Open"] == "Si"]
repetits = repe['id'][repe['id'].duplicated()].unique()
print(repetits)

[]
